# MORPC Insights - Distributed Energy Resources (DER)

## Overview

The Public Utilities Commission of Ohio maintains a [database](https://maps.puco.ohio.gov/arcgis/rest/services/electric/Distributed_Energy_Resources/MapServer/0/) containing the locations and attributes of distributed energy resources (DER) facilities and an associated [dashboard](https://maps.puco.ohio.gov/portal/apps/dashboards/ef2586cbf54b42cd8f5af3cf5c5da296). The dashboard provides the following notes:
  - A distributed energy resource (DER) is a source of electric power that is not directly connected to a bulk power system. DER includes both generators and energy storage technologies capable of exporting active power to the electric grid.
  - Energy Storage Capacity is reflective of standalone energy storage systems, not hybrid systems where capacity is already reported/captured in the generating units.
  - "Other" Fuel Types include Waste Gas, Biofuel, Diesel, Natural Gas/Propane, Coal, Cogeneration, and Hydro.
  
This notebook produces a tileset that includes a summary of DER facilities for the MORPC 15-county region and the counties and communities therein. This notebook is the final stage in a pipeline that fetches and standardizes the DER facility data (see [morpc-renewenergyfacilities-standardize](https://github.com/morpc/morpc-renewenergyfacilities-standardize)), and summarizes summarizes it by geography (see [morpc-renewenergyfacilities-summarize](https://github.com/morpc/morpc-renewenergyfacilities-summarize)).

The process defined herein is identified as process ID #65 in the the [Master Document List](https://morpc1.sharepoint.com/:x:/s/GISteam/EfC4j3HhohZCrSZzxJdyt5cBFEqVD7zHick8ZW0INqgCYA?e=Zc2yWF). This process is primarily intended for use in workflow ID #96.  References to identifiers in the master document list will subsequently be denoted by a number in brackets, e.g. [65].

This process is dependent on upstream processes.  See the "Prerequisites" section below.

## Prerequisites

  1. Clone the [morpc-renewenergyfacilities-summarize](https://github.com/morpc/morpc-renewenergyfacilities-summarize) repository and ensure that the paths are correct in the "Define inputs" section below.
  2. Execute the morpc-renewenergyfacilities-summarize workflow (including any upstream workflows as needed) to ensure that the outputs captured in the repository are up to date.
  3. Clone the [morpc-common](https://github.com/morpc/morpc-common) repository and adjust the path to morpc-common/morpc.py in the "Load required libraries" section below (if necessary).
  4. Clone the [morpc-geos-collect](https://github.com/morpc/morpc-geos-collect) repository and ensure that the paths are correct in the "Define inputs" section below.
  5. Clone the [morpc-lookup](https://github.com/morpc/morpc-lookup) repository and ensure that the paths are correct in the "Define inputs" section below.

## Setup

### Load required libraries

In [1]:
import pandas as pd
import frictionless
import math
import os
import sys
import json
import datetime
import textwrap
import matplotlib
from matplotlib import pyplot as plt
sys.path.append(os.path.normpath("../morpc-common"))
import morpc

### User-specified parameters

Adjust these as needed.

In [2]:
# YEAR_RANGE is a two-element list of integers that specifies the range of years for facility openings 
# to include in the output.  The first element is the beginning year in the range and the second element
# is the final year.
YEAR_RANGE = [2000, 2024]

### Static parameters

Typically these should not be adjusted.

In [3]:
# A copy of all input data will be archived in the location specified for INPUT_DIR
INPUT_DIR = os.path.normpath("./input_data")
print("Input data will be archived in directory: {}".format(INPUT_DIR))

# Output data produced by the script will stored in the location specified for OUTPUT_DIR
OUTPUT_DIR = os.path.normpath("./output_data")
print("Output data will be stored in directory: {}".format(OUTPUT_DIR))

# Charts produced by the script as images and Excel files will be stored in a subdirectory
# of OUTPUT_DIR with the name specified in CHART_DIRNAME
CHART_DIRNAME = "charts"
print("Charts will be stored in directory: {}".format(os.path.join(OUTPUT_DIR, CHART_DIRNAME)))

Input data will be archived in directory: input_data
Output data will be stored in directory: output_data
Charts will be stored in directory: output_data/charts


### Define inputs

#### Create input data directory

Create input data directory if it doesn't exist.

In [4]:
inputDir = os.path.normpath(INPUT_DIR)
if not os.path.exists(inputDir):
    os.makedirs(inputDir)

#### Summarized DER facilities data [408]

Table containing a summary of DER facilities and output capacity by geography by year. 

In [5]:
DER_INPUT_TABLE_RESOURCE = os.path.normpath("../morpc-renewenergyfacilities-summarize/output_data/morpc-renewenergyfacilities-der-long.resource.yaml")
print("Resource file: {}".format(DER_INPUT_TABLE_RESOURCE))

Resource file: ../morpc-renewenergyfacilities-summarize/output_data/morpc-renewenergyfacilities-der-long.resource.yaml


#### Geography lookup table [375]

Lookup table providing attributes and identifiers for Central Ohio geographies.

In [6]:
GEOS_LOOKUP_TABLE_RESOURCE = os.path.normpath("../morpc-geos-collect/output_data/morpc-geos-lookup.resource.yaml")
print("Resource file: {}".format(GEOS_LOOKUP_TABLE_RESOURCE))

Resource file: ../morpc-geos-collect/output_data/morpc-geos-lookup.resource.yaml


#### MORPC member list [122]

List of Central Ohio cities, villages, townships, and counties including indications whether each is or is not a MORPC member.

In [7]:
MEMBERS_DATA_PATH = "../morpc-lookup/Member_List.xlsx"
MEMBERS_SHEET = "Current Year Members"
MEMBERS_SCHEMA_PATH = "../morpc-lookup/Member_List_schema.json"
print("Data: {}, sheet '{}'".format(MEMBERS_DATA_PATH, MEMBERS_SHEET))
print("Schema: {}".format(MEMBERS_SCHEMA_PATH))

Data: ../morpc-lookup/Member_List.xlsx, sheet 'Current Year Members'
Schema: ../morpc-lookup/Member_List_schema.json


### Define outputs

#### Create output data directory

Create output data directory if it doesn't exist.

In [8]:
outputDir = os.path.normpath(OUTPUT_DIR)
if not os.path.exists(outputDir):
    os.makedirs(outputDir)   

Create the subdirectory to contain the charts if it doesn't exist.

In [9]:
chartDir = os.path.join(outputDir, CHART_DIRNAME)
if not os.path.exists(chartDir):
    os.makedirs(chartDir)    

#### DER facilities by geography by year [408]

Long-form table of DER facilities and capacity by geography by year intended to feed an [ArcGIS Dashboard](https://www.arcgis.com/apps/dashboards/3f2b48c930294cfda824567333f001fd).

In [10]:
FACILITIES_TABLE_FILENAME = "renewenergy-der-long.csv"
FACILITIES_TABLE_PATH = os.path.join(outputDir, FACILITIES_TABLE_FILENAME)
FACILITIES_TABLE_SCHEMA_PATH = FACILITIES_TABLE_PATH.replace(".csv",".schema.yaml")
FACILITIES_TABLE_RESOURCE_PATH = FACILITIES_TABLE_PATH.replace(".csv",".resource.yaml")
print("Data: {}".format(FACILITIES_TABLE_PATH))
print("Schema: {}".format(FACILITIES_TABLE_SCHEMA_PATH))
print("Resource file: {}".format(FACILITIES_TABLE_RESOURCE_PATH))

Data: output_data/renewenergy-der-long.csv
Schema: output_data/renewenergy-der-long.schema.yaml
Resource file: output_data/renewenergy-der-long.resource.yaml


## Prepare input data

### Load geography lookup table

Load the data from the source location, creating an archival copy in the input directory defined above.  Validate the data against the resource file and the schema to ensure that it complies with the schema and has not been altered.

In [11]:
(geosRaw, geosRawResource, geosRawSchema) = morpc.frictionless_load_data(
    GEOS_LOOKUP_TABLE_RESOURCE, 
    validate=True, 
    archiveDir=inputDir
)

morpc.load_frictionless_data | INFO | Loading Frictionless Resource file at location ../morpc-geos-collect/output_data/morpc-geos-lookup.resource.yaml
morpc.load_frictionless_data | INFO | Copying data, resource file, and schema to directory input_data
morpc.load_frictionless_data | INFO | --> Data file: input_data/morpc-geos-lookup.csv
morpc.load_frictionless_data | INFO | --> Resource file: input_data/morpc-geos-lookup.resource.yaml
morpc.load_frictionless_data | INFO | --> Schema file: input_data/morpc-geos-lookup.schema.yaml
morpc.load_frictionless_data | INFO | Validating resource including data and schema.
morpc.frictionless_validate_resource | INFO | Validating resource on disk (including data and schema). This may take some time.
morpc.frictionless_validate_resource | INFO | Resource is valid
morpc.load_frictionless_data | INFO | Loading data.
frictionless_cast_field_types | INFO | Casting field GEOIDFQ as type string.
frictionless_cast_field_types | INFO | Casting field GEOID 

Inspect the data.

In [12]:
geosRaw.head()

,GEOIDFQ,GEOID,SUMLEVEL,GEOTYPE,NAME,SOURCE,STATEFP,COUNTYFP,COUSUBFP,PLACEFP,TRACTCE,CLASSFP,MUNITYPE,PLACECOMBO
0,0500000US39041,39041,050,COUNTY,Delaware,CENSUS,39,041,<NA>,<NA>,<NA>,H1,<NA>,<NA>
1,0500000US39045,39045,050,COUNTY,Fairfield,CENSUS,39,045,<NA>,<NA>,<NA>,H1,<NA>,<NA>
2,0500000US39047,39047,050,COUNTY,Fayette,CENSUS,39,047,<NA>,<NA>,<NA>,H1,<NA>,<NA>
3,0500000US39049,39049,050,COUNTY,Franklin,CENSUS,39,049,<NA>,<NA>,<NA>,H1,<NA>,<NA>
4,0500000US39073,39073,050,COUNTY,Hocking,CENSUS,39,073,<NA>,<NA>,<NA>,H1,<NA>,<NA>


Create a working copy.

In [13]:
geos = geosRaw.copy()

### Load summarized DER facility data from upstream workflows

Load the data from the source location, creating an archival copy in the input directory defined above.  Validate the data against the resource file and the schema to ensure that it complies with the schema and has not been altered.

In [14]:
(facilitiesRaw, facilitiesRawResource, facilitiesRawSchema) = morpc.frictionless_load_data(DER_INPUT_TABLE_RESOURCE, validate=True, archiveDir=inputDir)

morpc.load_frictionless_data | INFO | Loading Frictionless Resource file at location ../morpc-renewenergyfacilities-summarize/output_data/morpc-renewenergyfacilities-der-long.resource.yaml
morpc.load_frictionless_data | INFO | Copying data, resource file, and schema to directory input_data
morpc.load_frictionless_data | INFO | --> Data file: input_data/morpc-renewenergyfacilities-der-long.csv
morpc.load_frictionless_data | INFO | --> Resource file: input_data/morpc-renewenergyfacilities-der-long.resource.yaml
morpc.load_frictionless_data | INFO | --> Schema file: input_data/morpc-renewenergyfacilities-der-long.schema.yaml
morpc.load_frictionless_data | INFO | Validating resource including data and schema.
morpc.frictionless_validate_resource | INFO | Validating resource on disk (including data and schema). This may take some time.
morpc.frictionless_validate_resource | INFO | Resource is valid
morpc.load_frictionless_data | INFO | Loading data.
frictionless_cast_field_types | INFO | Ca

Inspect the data.

In [15]:
facilitiesRaw.head()

,NAME,GEOIDFQ,SUMLEVEL,YEAR,METRIC,FUEL_TYPE,VALUE
0,15-County Region,M010000US001,M01,2000,Facilities,Biofuel,0.0
1,15-County Region,M010000US001,M01,2001,Facilities,Biofuel,0.0
2,15-County Region,M010000US001,M01,2002,Facilities,Biofuel,0.0
3,15-County Region,M010000US001,M01,2003,Facilities,Biofuel,0.0
4,15-County Region,M010000US001,M01,2004,Facilities,Biofuel,0.0


Create a working copy.

In [16]:
facilities = facilitiesRaw.copy()

### MORPC member list

The output data will include all communities for whom data has not been suppressed, however we will only show data for MORPC members in the platform.

Load the member table.

In [17]:
membersRaw = pd.read_excel(MEMBERS_DATA_PATH, sheet_name=MEMBERS_SHEET)
membersRaw.head()

,PLACECOMBO,PlaceLink,Local Member,County Member,Contains Any Member,MPO Member,MPO County,CORPO County,Model Area,FCPH Service Area,County,Place Name,GovType,GEOID
0,DELAWARE_ASHLEY_VILLAGE,4.0,0.0,1,True,1,1,0,1,0,Delaware,Ashley,Village,1550000US3902582041
1,DELAWARE_BERKSHIRE_TOWNSHIP,113.0,0.0,1,True,1,1,0,1,0,Delaware,Berkshire,Township,0700000US390410577499999
2,DELAWARE_BERLIN_TOWNSHIP,114.0,0.0,1,True,1,1,0,1,0,Delaware,Berlin,Township,0700000US390410578899999
3,DELAWARE_BROWN_TOWNSHIP,115.0,0.0,1,True,1,1,0,1,0,Delaware,Brown,Township,0700000US390410942899999
4,DELAWARE_COLUMBUS_CITY,22.0,1.0,1,True,1,1,0,1,0,Delaware,Columbus,City,1550000US3918000041


Load the schema.

In [18]:
membersSchema = morpc.load_avro_schema(MEMBERS_SCHEMA_PATH)

Verify that the fields are all the expected types.

In [19]:
members = morpc.cast_field_types(membersRaw, membersSchema)

Casting field PLACECOMBO as type string.
Casting field PlaceLink as type string.
Casting field Local Member as type boolean.
Casting field County Member as type boolean.
Casting field Contains Any Member as type boolean.
Casting field MPO Member as type boolean.
Casting field MPO County as type boolean.
Casting field CORPO County as type boolean.
Casting field Model Area as type boolean.
Casting field FCPH Service Area as type boolean.
Casting field County as type string.
Casting field Place Name as type string.
Casting field GovType as type string.
Casting field GEOID as type string.


Extract only the communities which are themselves a member.

In [20]:
members = members.loc[members["Local Member"] == True].copy()

The records in the member table are all county parts.  For places we need to subsitute the GEOID for the full place rather than the county part.

In [21]:
members["PLACEFP"] = members["GEOID"].apply(lambda x:x[11:16])
members["COUSUBFP"] = members["GEOID"].apply(lambda x:x[14:19])
members["COUNTYID"] = members["County"].map(morpc.CONST_COUNTY_NAME_TO_ID)
members["GEOIDFQ"] = None
temp = members.loc[members["GovType"] == "Township"].copy()
temp["GEOIDFQ"] = temp["GEOID"]
members.update(temp)
temp = members.loc[members["GovType"] != "Township"].copy()
temp["GEOIDFQ"] = "1600000US39" + temp["PLACEFP"]
members.update(temp)

Now extract just the list of member GEOIDs.  The steps above likely produced duplicate records for places, so extract only the unique GEOIDs.

In [22]:
memberList = list(members["GEOIDFQ"].unique())

We also need to append the list of counties.  We will include data for all counties regardless of membership status.

In [23]:
memberList += ["0500000US{}".format(morpc.CONST_COUNTY_NAME_TO_ID[x]) for x in morpc.CONST_REGIONS['REGION15']]

Finally we need to append the ID for the MORPC region.

In [24]:
memberList.append("M010000US001")

## Transform data to format required by Insights platform

### Extract data for specified range of years

Extract facilities which opened in the year range specified in the "User-specified parameters" section.

In [25]:
facilities = facilities.loc[facilities["YEAR"].isin(range(YEAR_RANGE[0], YEAR_RANGE[1]+1))].copy()

### Standardize geography names and identifiers

Assign a human-readable geography type for each geography based on its SUMLEVEL code.

In [26]:
facilities["GEOTYPE"] = facilities["SUMLEVEL"].map(morpc.HIERARCHY_STRING_LOOKUP)

Drop the geography names that were included in the facilities data and replace them with the standard names found in MORPC's geography lookup page. Also add the county FIPS code for the county that contains the geography.  Note that the FIPS code will be null for geographies that span multiple counties.

In [27]:
facilities = facilities.drop(columns="NAME").merge(geos[["GEOIDFQ","COUNTYFP","NAME"]], on="GEOIDFQ")

Convert the county FIPS code to a complete county GEOID by prepending the Ohio state FIPS code ("39").

In [28]:
facilities["COUNTYFP"] = "39" + facilities["COUNTYFP"]

Create a field for the county name and look up the county name using the county GEOID.

In [29]:
facilities["COUNTY"] = facilities["COUNTYFP"].map(morpc.CONST_COUNTY_ID_TO_NAME)

For records representing the non-incorporated portions of townships (SUMLEVEL 070), append the word "Township" to the name followed by the name of the county that contains it in parentheses.  This is necessary because township names are sometimes reused in multiple counties. 

In [30]:
temp = facilities.loc[facilities["SUMLEVEL"] == "070"].copy()
temp["NAME"] = temp["NAME"] + " Township (" + temp["COUNTY"] + ")"
facilities.update(temp, overwrite=True, errors="ignore")

For records representing whole counties (SUMLEVEL 050), append the word "County" to the name.  This is necessary because sometimes incorporated places or townships have the same names as counties (e.g. Delaware City and Delaware County).

In [31]:
temp = facilities.loc[facilities["SUMLEVEL"] == "050"].copy()
temp["NAME"] = temp["NAME"] + " County"
facilities.update(temp, overwrite=True, errors="ignore")

### Pivot the data to semi-wide form

Extract only the fields we require.

In [32]:
facilities = facilities.filter(items=["GEOIDFQ","NAME","GEOTYPE","YEAR","METRIC","FUEL_TYPE","VALUE"], axis="columns")

Pivot the data to wide format such that each record represents the generating capacity and number of facilities using a particular fuel type that came online in a particular geography and year.

In [33]:
facilities = facilities.pivot(index=["GEOIDFQ","NAME","GEOTYPE","YEAR","FUEL_TYPE"], columns="METRIC", values="VALUE").reset_index()
facilities.columns.name = None

### Change the units of generating capacity from kilowatts to watts

The generating capacity is currently expressed in kilowatts.  This save some storage space and makes the data easier to parse by eye, but it leads to confusing scaling of charts in the Insights platform. To make the charts clearer, we'll convert the capacity values to watts and then represent them in the charts using common suffixes, i.e. 10k watts or 1M watts.

In [34]:
facilities["Capacity"] = facilities["Capacity"] * 1000

### Reformat the data to comply with the output schema

Load the schema for the output data.

In [35]:
facilitiesSchema = morpc.frictionless_load_schema(FACILITIES_TABLE_SCHEMA_PATH)
facilitiesSchema

{'fields': [{'name': 'GEOIDFQ',
             'type': 'string',
             'description': 'Unique identifier for the geography as issued by '
                            'MORPC.  These are identical to fully-qualified '
                            'Census-issued GEOIDs for Census geographies.'},
            {'name': 'NAME',
             'type': 'string',
             'description': 'Name of the geography.'},
            {'name': 'GEOTYPE',
             'type': 'string',
             'description': 'Human readable string with describes the summary '
                            'level (geography type) for which the GEOID '
                            'applies.  The combination of GEOTYPE and GEOID '
                            'uniquely identify the geography for the record.'},
            {'name': 'YEAR',
             'type': 'integer',
             'description': 'Four-digit year in which the Distributed Energy '
                            'Resource facility was registered.'},
      

Rename the variables to match the schema.

In [36]:
facilities = facilities.rename(columns={
        "Capacity":"CAPACITY",
        "Facilities":"FACILITIES"
})

Re-cast the variables to the types specified in the schema.

In [37]:
facilities = morpc.cast_field_types(facilities, facilitiesSchema)

frictionless_cast_field_types | INFO | Casting field GEOIDFQ as type string.
frictionless_cast_field_types | INFO | Casting field NAME as type string.
frictionless_cast_field_types | INFO | Casting field GEOTYPE as type string.
frictionless_cast_field_types | INFO | Casting field YEAR as type integer.
frictionless_cast_field_types | INFO | Casting field FUEL_TYPE as type string.
frictionless_cast_field_types | INFO | Casting field FACILITIES as type integer.
frictionless_cast_field_types | INFO | Casting field CAPACITY as type number.


Extract only the variables required by the schema.

In [38]:
facilities = facilities.filter(items=facilitiesSchema.field_names, axis="columns")

Sort the data by geography type, then geography name, then county, then year, then fuel type.

In [39]:
facilities = facilities.sort_values(by=["GEOTYPE","NAME","YEAR","FUEL_TYPE"])

Inspect the data.

In [40]:
facilities.head()

,GEOIDFQ,NAME,GEOTYPE,YEAR,FUEL_TYPE,FACILITIES,CAPACITY
0,0500000US39041,Delaware County,COUNTY,2000,Biofuel,0,0.0
1,0500000US39041,Delaware County,COUNTY,2000,Energy Storage,0,0.0
2,0500000US39041,Delaware County,COUNTY,2000,Natural Gas/Propane,0,0.0
3,0500000US39041,Delaware County,COUNTY,2000,Solar,0,0.0
4,0500000US39041,Delaware County,COUNTY,2000,Waste Gas,0,0.0


## Export data

Export the data to a CSV file.

In [41]:
facilities.to_csv(FACILITIES_TABLE_PATH, index=False)

## Create resource file for exported data

Create a [Frictionless Resource file](https://specs.frictionlessdata.io/tabular-data-resource/) for the exported data. The resource file associates the schema with the CSV file and captures key metadata about the CSV file including the filesize (bytes) and MD5 checksum (hash).  This facilities validation and integrity checking of the CSV file.  The CSV file is automatically validated after the resource file is created.

In [42]:
facilitiesResource = morpc.frictionless_create_resource(FACILITIES_TABLE_FILENAME, 
    resourcePath=FACILITIES_TABLE_RESOURCE_PATH,
    title="MORPC Insights | Distributed Energy Resources Facilities by Year", 
    name="renewenergy_der", 
    description="Count and generation capacity of Central Ohio Distributed Energy Resources facilites which opened in each year according to data maintained by the Public Utilities Commission of Central Ohio.",
    writeResource=True,
    validate=True
)
facilitiesResource

morpc.frictionless_create_resource | INFO | Format not specified. Using format derived from data file extension: csv
morpc.frictionless_create_resource | INFO | Schema path not specified. Using path derived from data file path: renewenergy-der-long.schema.yaml
morpc.frictionless_create_resource | INFO | Writing Frictionless Resource file to output_data/renewenergy-der-long.resource.yaml
morpc.frictionless_create_resource | INFO | Validating resource on disk.
morpc.frictionless_validate_resource | INFO | Validating resource on disk (including data and schema). This may take some time.
morpc.frictionless_validate_resource | INFO | Resource is valid


{'name': 'renewenergy_der',
 'type': 'table',
 'title': 'MORPC Insights | Distributed Energy Resources Facilities by Year',
 'description': 'Count and generation capacity of Central Ohio Distributed '
                'Energy Resources facilites which opened in each year '
                'according to data maintained by the Public Utilities '
                'Commission of Central Ohio.',
 'profile': 'data-resource',
 'path': 'renewenergy-der-long.csv',
 'scheme': 'file',
 'format': 'csv',
 'mediatype': 'text/csv',
 'hash': '421c5d72c703ec867877e1ba279f8003',
 'bytes': 3855236,
 'schema': 'renewenergy-der-long.schema.yaml'}

## Generate static charts

This section will generate static charts in SVG (scalable vector graphics) and Excel format.  The SVG charts will be displayed on the tiles in the Insights platform.  Both the SVG charts and the Excel charts will be made available via GitHub for general purpose usage.

### Prepare to create charts

First delete any existing contents in the chart directory. This ensures that we are not left with any stale content in the event that no new chart is produced for a geography during this run.

In [112]:
for f in os.scandir(chartDir):
    os.remove(f)

Load a standard color set for the chart elements.

In [44]:
colorset = json.loads(json.dumps(morpc.CONST_COLOR_CYCLES["matplotlib"]))

### Create charts of number of facilities coming online by year

In [113]:
%matplotlib agg  
# The preceeding line disables display of matplotlib charts in the notebook

# Create a list to accumulate geographies for which a thumbnail is generated
platformIncludeList = []

# Iterate over each geography in data set
for geoid in facilities["GEOIDFQ"].unique():
    # If the geography is not a MORPC member, skip it. The platform only features members.
    if(not geoid in memberList):
        continue
    
    # Extract the data for a single geography.  If the resulting dataframe is empty, this
    # means there are no records for that geography.  In that case, skip this geography
    # and restart the loop with the next geography.
    temp = facilities.loc[facilities["GEOIDFQ"] == geoid].copy()
    if(temp.empty):
        continue
    
    # Add the geography to the list to be included in the platform
    platformIncludeList.append(geoid)

    # Generate chart title which includes the geography name
    geoName = temp.iloc[0]["NAME"]
    title = "Distributed Energy Resources Facilities by Year Opened - {}".format(geoName)
    
    # Define the x-axis and y-axis labels. Both axes are intuitive thanks to the chart title so
    # no labels are needed
    xlabel = None
    ylabel = None
    
    # Drop the geography name and type, which are not needed in the chart
    temp = temp.filter(items=["YEAR","FUEL_TYPE","FACILITIES"], axis="columns")
    
    # Make the variable names nicer looking
    temp = temp.rename(columns={
        "YEAR":"Open year",
        "FUEL_TYPE":"Fuel type",
        "FACILITIES":"Facilities"
    })
    
    # Pivot to wide format
    temp = temp.pivot(index="Open year", columns="Fuel type", values="Facilities")
    temp.columns.name = None
    
    ## Create and annotate the plot
    # Specify a width of 8 inches and an aspect ratio of 16:9
    PLOTWIDTH = 8
    fig,ax = plt.subplots(figsize=(PLOTWIDTH,PLOTWIDTH/16*9))

    # Create a stacked bar chart with years on the x-axis and number of facilities 
    # on the y-axis
    temp.plot.bar(ax=ax, stacked=True, color=colorset)

    # Apply the title and axis labels specified above to the chart
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    # Create the legend. Wrap the legend labels to 15 characters to keep the legend compact
    # Place the legend to the right of the chart.
    handles, labels = ax.get_legend_handles_labels()
    labels = [textwrap.fill(label, 15) for label in labels]
    legend = ax.legend(handles[::-1], labels[::-1], loc='center left', bbox_to_anchor=(1, 0.5), labelspacing=1)
    
    # Add gridlines to the chart and ensure that they are drawn beneath the bars
    ax.grid(visible=True, color="lightgrey")
    ax.set_axisbelow(True)

    # Tell matplotlib to use only integer multiples for the y-axis ticks.  Otherwise we end up with repeated
    # tick labels when data values are small.
    ax.get_yaxis().set_major_locator(matplotlib.ticker.MaxNLocator(nbins="auto", steps=[1, 2, 5, 10]))

    ax.set_yticks(sorted(list(set([int(x) for x in ax.get_yticks()]))))
    
    # Format the y-axis labels as integers with comma separators
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    
    # Format the y-axis labels using engineering notation (k, M)
    #ax.get_yaxis().set_major_formatter(matplotlib.ticker.EngFormatter())
    
    # Save the figure to disk as an SVG file
    ax.figure.savefig(os.path.join(chartDir, "facilities-{0}-{1}.svg".format(geoName.replace(" ","").replace("(","").replace(")",""), geoid)), bbox_extra_artists=(legend,), bbox_inches='tight')

    # Destroy the matplotlib chart in memory.
    plt.close(ax.figure)

    # Create a blank Excel document to hold the data table and chart
    writer = pd.ExcelWriter(os.path.join(chartDir, "facilities-{0}-{1}.xlsx".format(geoName.replace(" ","").replace("(","").replace(")",""), geoid)), engine='xlsxwriter')
    
    # Configure the presentation of the data table, as required by morpc.data_chart_to_excel()
    dataOptions = {
        "numberFormat": {
            'Open year': "0",
            'Biofuel': "#,##0",
            'Energy Storage': "#,##0",
            'Natural Gas/Propane': "#,##0",
            'Solar': "#,##0",
            'Waste Gas': "#,##0",
            'Wind': "#,##0"
        },
        "columnWidth": 20
    }

    # Configure the presentation of the chart, as required by morpc.data_chart_to_excel()
    chartOptions = {
        "subtype":"stacked",
        "colors": colorset,
        "titles": {
            "chartTitle": title,
            "xTitle": xlabel,
            "yTitle": ylabel
        },
        "seriesOptions": [{"gap":100} for x in temp.columns],
        "xAxisOptions": {
            "num_font": {"size":14},
        },
        "yAxisOptions": {
            "num_font": {"size":14},
            "num_format": "#,##0",
        },
        "legendOptions":{
            "position":"bottom",
            "font":{"size":14}
        },
        "sizeOptions":{
            "x_scale":1.5,
            "y_scale":1.5
        }
    }

    # Add the data table and chart to the Excel document
    morpc.data_chart_to_excel(temp, writer, chartType="column", dataOptions=dataOptions, chartOptions=chartOptions)

    # Close the Excel document
    writer.close()    

# Reenable display of matplotlib charts in the notebook
%matplotlib inline

### Capacity

In [ ]:
%matplotlib agg

# Create a list to accumulate geographies for which a thumbnail is generated
platformIncludeList = []

# Iterate over each geography in data set
for geoid in facilities["GEOIDFQ"].unique():
    # If the geography is not a MORPC member, skip it. The platform only features members.
    if(not geoid in memberList):
        continue
    
    # Extract the data for a single geography
    temp = facilities.loc[facilities["GEOIDFQ"] == geoid].copy()

    if(temp.empty):
        continue
        
    platformIncludeList.append(geoid)

    # Generate a title string
    geoName = temp.iloc[0]["NAME"]
    title = "Distributed Energy Resources Capacity by Year Opened - {}".format(geoName)
    xlabel = None
    ylabel = "Kilowatts (kW)"
    
    # Drop the geography name and type
    temp = temp.filter(items=["YEAR","FUEL_TYPE","CAPACITY"], axis="columns")
    
    # Make the variable names nicer looking
    temp = temp.rename(columns={
        "YEAR":"Year",
        "FUEL_TYPE":"Fuel type",
        "CAPACITY":"Capacity"
    })

    # Pivot to wide format
    temp = temp.pivot(index="Year", columns="Fuel type").reset_index()
    
    # Create and annotate the plot
    PLOTWIDTH = 8
    fig,ax = plt.subplots(figsize=(PLOTWIDTH,PLOTWIDTH/16*9))

    temp.plot.bar(ax=ax, x="Year", y="Capacity", stacked=True, color=colorset)
    temp = facilities.loc[facilities["GEOIDFQ"] == geoid].copy()
    
    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_yticks([round(tick,0) for tick in ax.get_yticks()])
    
    handles, labels = ax.get_legend_handles_labels()
    labels = [textwrap.fill(label, 15) for label in labels]
    legend = ax.legend(handles[::-1], labels[::-1], loc='center left', bbox_to_anchor=(1, 0.5), labelspacing=1)
    ax.grid(visible=True, color="lightgrey")
    ax.set_axisbelow(True)
    
    # Format the y-axis labels as integers with comma separators
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
    
    # Save the figure to disk as an SVG file
    ax.figure.savefig(os.path.join(chartDir, "capacity-{}.svg".format(geoid)), bbox_extra_artists=(legend,), bbox_inches='tight')
    
    plt.close(ax.figure)

    excelData = temp[["YEAR","FUEL_TYPE","CAPACITY"]].pivot(index="YEAR", columns="FUEL_TYPE", values="CAPACITY")
    excelData = excelData.rename(columns={
        "CAPACITY":"Capacity"
    })
    excelData.index.name = "Open year"
    excelData.columns.name = None

    writer = pd.ExcelWriter(os.path.join(chartDir, "capacity-{}.xlsx".format(geoid)), engine='xlsxwriter')
    dataOptions = {
        "numberFormat": {
            'Open year': "0",
            'Biofuel': "#,##0.0",
            'Energy Storage': "#,##0.0",
            'Natural Gas/Propane': "#,##0.0",
            'Solar': "#,##0.0",
            'Waste Gas': "#,##0.0",
            'Wind': "#,##0.0"
        },
        "columnWidth": 20
    }
    chartOptions = {
        "subtype":"stacked",
        "colors": colorset,
        "titles": {
            "chartTitle": title,
            "xTitle": xlabel,
            "yTitle": ylabel
        },
        "seriesOptions": [{"gap":100} for x in excelData.columns],
        "xAxisOptions": {
            "num_font": {"size":14},
        },
        "yAxisOptions": {
            "num_font": {"size":14},
            "num_format": "#,##0",
        },
        "legendOptions":{
            "position":"bottom",
            "font":{"size":14}
        },
        "sizeOptions":{
            "x_scale":1.5,
            "y_scale":1.5
        }
    }
    morpc.data_chart_to_excel(excelData, writer, chartType="column", dataOptions=dataOptions, chartOptions=chartOptions)
    writer.close()    
    
%matplotlib inline

## Generate Insights catalog content

The content in the Insights platform is controlled by a catalog spreadsheet. Each tile to be displayed in the platform must have a record in the catalog.  This section will create the records for the tiles that display the alternative fuel station data.  Eventually this function will be performed by a separate staging script.

First specify the column names used in the catalog.

In [ ]:
columnNames=["TileID","TilesetID","GeographyType","GeographyName","Category","Headline","Commentary","ThumbnailURL","Contributor","Vintage","UpdateInterval","ShareURL","DataProductURL","MoreInformationURL"]

Create a new dataframe containing only the geographies for which thumbnail images were produced in the section above.

In [ ]:
catalog = facilities.loc[facilities["GEOIDFQ"].isin(platformIncludeList)].copy()

Extract only the metadata columns of interest and flatten the data to have only one record per geography. Rename the metadata fields to match the catalog fields.

In [ ]:
catalog = catalog.filter(items=["GEOIDFQ","NAME","GEOTYPE"], axis="columns") \
    .groupby("GEOIDFQ").first() \
    .reset_index() \
    .rename(columns={"NAME":"GeographyName","GEOTYPE":"GeographyType"})

Change the GeographyType values to match the schema of the catalog.

In [ ]:
catalog["GeographyType"] = catalog["GeographyType"].map({
    "REGION15":"Region",
    "COUNTY":"County",
    "COUNTY-TOWNSHIP-REMAINDER":"Community",
    "PLACE":"Community"
})

Populate some placeholder fields.

In [ ]:
catalog["TileID"] = None
catalog["TilesetID"] = "TBD-Facilities"
catalog["Category"] = None
catalog["Headline"] = "TBD"
catalog["Commentary"] = "TBD"

Generate the URL for the thumbnail images. These will be hosted in GitHub and will be indexed by GEOIDFQ.

In [ ]:
catalog["ThumbnailURL"] = catalog["GEOIDFQ"].apply(lambda geoid:"https://raw.githubusercontent.com/morpc-insights/renewenergy-der/refs/heads/main/output_data/charts/facilities-{}.svg".format(geoid))

Populate some other simple metadata.  Vintage in this case refers to the year that the content was published in Insights. This is to give readers an idea of how old it is.  UpdateInterval gives them an idea of when to expect the next version. ShareURL is a placeholder for now.

In [ ]:
catalog["Contributor"] = "Mid-Ohio Regional Planning Commission"
catalog["Vintage"] = str(datetime.date.today().year)
catalog["UpdateInterval"] = "annually"
catalog["ShareURL"] = None

Generate the data product URL.  This points to an ArcGIS Dashboard that accepts URL parameters.  GEOIDFQ is passed as a parameter to tell the app to load the data for a particular geography.

In [ ]:
catalog["DataProductURL"] = catalog["GEOIDFQ"].apply(lambda geoid:"https://www.arcgis.com/apps/dashboards/3f2b48c930294cfda824567333f001fd#geoid={}".format(geoid))

Generate the URLs that point to the extended commentary pages.  Default to a common page (population.pdf) hosted in GitHub.  Point to specific pages for the 15-county region and for each county.

In [ ]:
catalog["MoreInformationURL"] = "https://morpc1-my.sharepoint.com/:w:/g/personal/aporr_morpc_org/EYb3oBwdBFJNnHr_wHq-rUoBpiSYkdrVfFQ19rW7vJeh8Q?e=NTX9dX"

Extract only the required columns.

In [ ]:
catalog = catalog.filter(items=columnNames, axis="columns")

Inspect the listing.

In [ ]:
catalog.head()

In [ ]:
temp = catalog.copy()

In [ ]:
temp["ThumbnailURL"] = temp["ThumbnailURL"].str.replace("facilities-", "capacity-")

In [ ]:
temp["TilesetID"] = temp["TilesetID"].str.replace("Facilities", "Capacity")

In [ ]:
catalog = pd.concat([catalog, temp], axis="index")

Save the catalog to an Excel spreadsheet.

In [ ]:
catalog.to_excel("catalog.xlsx", index=False)

It is necessary to manually add these records to the master catalog or update the records already therein.  See the following file in GitHub:

https://github.com/morpc/morpc-insights/blob/main/catalog/morpc_insights_catalog.xlsx
